In [ ]:
# 0) Installing dependencies
!pip install -qU transformers datasets evaluate accelerate peft trl bitsandbytes wandb
!pip install -qU nvidia-ml-py3

# 1) Imports
import os
import pandas as pd
import torch
import numpy as np
import wandb
from datasets import load_dataset, Dataset
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
)
from peft import LoraConfig, get_peft_model, TaskType
import evaluate
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

# Logging into Weights & Biases
wandb.login()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 2) Loading & tokenizing AG News
base_model = "roberta-base"
raw = load_dataset("ag_news", split="train")
tokenizer = RobertaTokenizer.from_pretrained(base_model)

def preprocess(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=128,
    )


tok = raw.map(preprocess, batched=True, remove_columns=["text"])
tok = tok.rename_column("label", "labels")
tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

split = tok.train_test_split(test_size=640, seed=42)
train_ds, eval_ds = split["train"], split["test"]

# 3) PEFT / LoRA configuration
num_labels = raw.features["label"].num_classes
id2label = {i: name for i, name in enumerate(raw.features["label"].names)}

peft_config = LoraConfig(
    r=4,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    target_modules=["query", "value"],
    modules_to_save=[],
    task_type=TaskType.SEQ_CLS,
)


base = RobertaForSequenceClassification.from_pretrained(
    base_model, num_labels=num_labels, id2label=id2label
)
model = get_peft_model(base, peft_config)
model.to(device)

# 3a) Verify trainable params < 1M
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable} ({trainable/1e6:.3f}M)")
assert trainable < 1_000_000, "Trainable parameters exceed 1M—lower LoRA rank!"

# 4) Metrics
accuracy = evaluate.load("accuracy")
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return accuracy.compute(predictions=preds, references=p.label_ids)





wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Using device: cuda


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable params: 741124 (0.741M)


In [ ]:
# 5) TrainingArguments with W&B reporting
training_args = TrainingArguments(
    output_dir="results",
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=200,
    eval_steps=200,
    learning_rate=1e-5,
    lr_scheduler_type="cosine",
    warmup_steps=500,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=64,
    num_train_epochs=8,
    weight_decay=0.01,
    label_smoothing_factor=0.1,
    optim="adamw_torch",
    gradient_checkpointing=True,
    fp16=True,
    group_by_length=True,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=200,
    report_to="wandb",
    run_name="agnews-lora-roberta",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    seed=42,
)


# 6) Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)


# 7) Train & Evaluate
trainer.train()
res = trainer.evaluate()
print("Validation Accuracy:", res["eval_accuracy"])

# 8) Save fine‑tuned model & tokenizer
model.save_pretrained("./lora_roberta_agnews_final")
tokenizer.save_pretrained("./lora_roberta_agnews_final")

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
200,1.389800,1.386671,0.239063
400,1.381500,1.375006,0.421875
600,1.366900,1.358903,0.567187
800,1.349000,1.341583,0.581250
1000,1.331000,1.323202,0.750000
1200,1.312300,1.306762,0.759375
1400,1.296100,1.288070,0.781250
1600,1.277700,1.268805,0.800000
1800,1.254700,1.246599,0.831250
2000,1.234900,1.226496,0.837500


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn

Validation Accuracy: 0.88125


('./lora_roberta_agnews_final/tokenizer_config.json',
 './lora_roberta_agnews_final/special_tokens_map.json',
 './lora_roberta_agnews_final/vocab.json',
 './lora_roberta_agnews_final/merges.txt',
 './lora_roberta_agnews_final/added_tokens.json')

In [ ]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

# Load your pickled test set
raw = pd.read_pickle("test_unlabelled.pkl")


if isinstance(raw, Dataset):
    hf_test = raw
else:
    hf_test = Dataset.from_pandas(raw.reset_index(drop=True))

print("test set rows:", len(hf_test), "columns:", hf_test.column_names)

# 2) Tokenize (drop the original text column)
def tokenize_fn(ex):
    return tokenizer(
        ex["text"],
        padding="max_length",
        truncation=True,
        max_length=256,
    )

hf_test = hf_test.map(
    tokenize_fn,
    batched=True,
    remove_columns=["text"],
)
hf_test.set_format("torch", columns=["input_ids", "attention_mask"])

# DataLoader & Inference
loader = DataLoader(hf_test, batch_size=64)
model.to(device).eval()

all_preds = []
with torch.no_grad():
    for batch in tqdm(loader, desc="Inference"):
        batch = {k: v.to(device) for k, v in batch.items()}
        logits = model(**batch).logits
        all_preds.extend(torch.argmax(logits, dim=-1).cpu().numpy())

# Save submission.csv
submission = pd.DataFrame({
    "ID": np.arange(len(all_preds)),
    "Label": all_preds
})
submission.to_csv("submission.csv", index=False)
print("submission.csv rows:", len(submission))

test set rows: 8000 columns: ['text']


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Inference:   0%|          | 0/125 [00:00<?, ?it/s]

submission.csv rows: 8000


In [ ]:
#Uploaded to Github Repository: https://github.com/greeeshmaaa/Deep-Learning-Spring-2025-Finetuning-with-LoRA/tree/main